In [1]:
class Player:
    def __init__(self, name):
        self.name = name
        self.wins = 0

In [2]:
class Throw:

    def __init__(self, name, defeated_throws):
        self.name = name
        self.defeated_throws = defeated_throws

    def can_defeat(self, throw):
        if throw.name in self.defeated_throws:
            return True
        else:
            return False

In [27]:
import random

def game():
    print_header()
    throws = build_3_possible_throws()
    name = get_player1_name()
    player1 = Player(name)
    player2 = Player('Computer')
    game_loop(player1, player2, throws)


def print_header():
    print('--------------------------------------------------')
    print('               Rock, Paper, Scissors              ')
    print('--------------------------------------------------')
    return


def build_3_possible_throws():
    Rock = Throw('Rock', ['Scissors'])
    Paper = Throw('Paper', ['Rock'])
    Scissors = Throw('Scissors',  ['Paper'])
    return {'Rock': Rock, 'Paper': Paper, 'Scissors': Scissors}


def get_player1_name():
    name = input('What is your name? ')
    print()
    return str(name.strip())


def get_player1_throw(throws):
    pick = input('Pick [r]ock, [p]aper, or [s]cissors: ')
    print()

    if pick not in ['r', 'p', 's'] or pick is None:
        print("** Please enter a single letter: " +
               "'r' for rock, 'p' for paper, or 's' for scissiors **")
        return get_player1_throw(throws)
    elif pick == 'r':
        return throws['Rock']
    elif pick == 'p':
        return throws['Paper']
    elif pick == 's':
        return throws['Scissors']


def game_loop(player1, player2, throws):
    count = 1
    decisive_rounds = 0

    while decisive_rounds < 3 and player1.wins < 2 and player2.wins < 2:
        player2_throw = random.choice([throw for throw in throws.values()])
        player1_throw = get_player1_throw(throws)

        player1_win = player1_throw.can_defeat(player2_throw)
        player2_win = player2_throw.can_defeat(player1_throw)

        if player1_win:
            winner = player1
        elif player2_win:
            winner = player2
        elif not player1_win and not player2_win:
            winner = None

        print(f'Round {count}')
        print(f'_____________')
        print(f'{player1.name} threw a {player1_throw.name}')
        print(f'{player2.name} threw a {player2_throw.name}')
        print()

        try:
            print(f'The winner for this round is {winner.name}')
            winner.wins += 1
            decisive_rounds += 1
        except (AttributeError):
            print(f'This round had no winner and is a tie round')

        print()
        count += 1

    if player1.wins > player2.wins:
        overall_winner = player1
    elif player2.wins > player1.wins:
        overall_winner = player2

    print(f'{overall_winner.name} is the winner!')
    print(('{0} won {1} out of {2} rounds that did not end in tie'
           .format(overall_winner.name, overall_winner.wins, decisive_rounds)))

In [28]:
throws = build_3_possible_throws()
result = get_player1_throw(throws)
result.name

Pick [r]ock, [p]aper, or [s]cissors: r



'Rock'

In [29]:
# game()

In [31]:
from unittest.mock import patch
import pytest
import ipytest
import ipytest.magics

ipytest.config.rewrite_asserts = True
__file__ = 'RPS_Notebook.ipynb'


def test_Throw():
    item = 'Dragon'
    can_defeat = ['Eagle', 'Sheep']
    dragon = Throw(item, can_defeat)
    eagle = Throw('Eagle',[])
    sheep = Throw('Sheep',[])
    kracken = Throw('Kracken',[])
    assert dragon.name == 'Dragon'
    assert dragon.can_defeat(kracken) == False
    assert dragon.can_defeat(sheep) == True
    assert dragon.can_defeat(eagle) == True


def test_Player():
    name = "Sally"
    player = Player("Sally")
    assert player.name == "Sally"
    assert player.wins == 0
    player.wins += 1
    assert player.wins == 1

def test_print_header(capfd):  #capfd
    print_header()
    output, err = capfd.readouterr()
    expected = ['--------------------------------------------------',
                '               Rock, Paper, Scissors              ',
                '--------------------------------------------------']
    output = output.split('\n')
    for out, exp in zip(output, expected):
        assert out == exp

def test_build_3_possible_throws():
    throws = build_3_possible_throws()
    assert throws['Rock'].name == 'Rock'
    assert throws['Rock'].defeated_throws == ['Scissors']
    assert throws['Paper'].name == 'Paper'
    assert throws['Paper'].defeated_throws == ['Rock']
    assert throws['Scissors'].name == 'Scissors'
    assert throws['Scissors'].defeated_throws == ['Paper']

@patch("builtins.input", side_effect = ["Harry the Great  "])
def test_get_player1_name(inp):
    name = get_player1_name()
    assert name == "Harry the Great"

# My fixture did not work; had to fake it
# @pytest.fixture
# def throws():
#     return build_3_possible_throws()
    

@patch("builtins.input", side_effect = ['r','p','s'])
def test_get_player1_throw_valid(inp, throws = build_3_possible_throws()):
    expected = 'Rock'
    assert get_player1_throw(throws).name == expected
    expected = 'Paper'
    assert get_player1_throw(throws).name == expected
    expected = 'Scissors'
    assert get_player1_throw(throws).name == expected
    

@patch("builtins.input", side_effect = ['rps', 'r', None, 'r'])
def test_get_player1_throw_invalid(inp, capfd, throws = build_3_possible_throws()):
    get_player1_throw(throws)                    
    output, _ = capfd.readouterr()
    expected = ("** Please enter a single letter: " +
               "'r' for rock, 'p' for paper, or 's' for scissiors **")
    output = output.split('\n')
    output = output[1] + output[2]
    print(f'output = {output}')
    assert output == expected
    
    
# Not really sure I can patch my random.choice function since it needs to return 
# a specific throw object
# @patch("builtins.input", side_effect = [])
# @patch.object(random, 'choice')
# def test_game():
#     pass
    
ipytest.run('-v')

========================================= test session starts ==========================================
platform darwin -- Python 3.7.0, pytest-3.8.2, py-1.7.0, pluggy-0.7.1 -- /anaconda3/bin/python
cachedir: .pytest_cache
rootdir: /Users/NewEarth/Documents/Projects/ANewTraining/Python/HundredDaysCode/My100Days, inifile:
plugins: cov-2.6.0
collecting ... collected 8 items

RPS_Notebook.py::test_game <- <ipython-input-30-aa0beccc08a4> PASSED                             [ 12%]
RPS_Notebook.py::test_Throw <- <ipython-input-31-94cd4e669dec> PASSED                            [ 25%]
RPS_Notebook.py::test_Player <- <ipython-input-31-94cd4e669dec> PASSED                           [ 37%]
RPS_Notebook.py::test_print_header <- <ipython-input-31-94cd4e669dec> PASSED                     [ 50%]
RPS_Notebook.py::test_build_3_possible_throws <- <ipython-input-31-94cd4e669dec> PASSED          [ 62%]
RPS_Notebook.py::test_get_player1_name <- <ipython-input-31-94cd4e669dec> PASSED                 [ 75%]